In [53]:
import numpy as np
import pandas as pd
import math
from zipfile import ZipFile
from urllib.request import urlretrieve
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import StringLookup
from pathlib import Path

In [54]:
# loading data from CSV file
r_cols = ['userId', 'movieId', 'rating', 'timestamp']
m_cols=['movieId', 'title', 'genres']
movies = pd.read_csv('F:\MS-Research\Group RS\Datasets\ml-1m/movies.dat', sep='::', names=m_cols, encoding='ISO-8859-1')
ratings = pd.read_csv('F:\MS-Research\Group RS\Datasets\ml-1m/ratings.dat', sep='::',  names=r_cols, encoding='ISO-8859-1')

F:\Anaconda3\envs\tf\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [55]:
df_combined = pd.merge(ratings, movies, on = 'movieId')

In [56]:
df_combined

,userId,movieId,rating,timestamp,title,genres
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
1000204,5949,2198,5,958846401,Modulations (1998),Documentary
1000205,5675,2703,3,976029116,Broken Vessels (1998),Drama
1000206,5780,2845,1,958153068,White Boys (1999),Drama
1000207,5851,3607,5,957756608,One Little Indian (1973),Comedy|Drama|Western


In [57]:
Mem_mat=pd.read_csv('mem_mat.csv')

In [58]:
Mem_mat

,Action,Adventure
0,0.526894,0.473106
1,0.520339,0.479661
2,0.502789,0.497211
3,0.519306,0.480694
4,0.462777,0.537223
...,...,...
6035,0.403684,0.596316
6036,0.398373,0.601627
6037,0.535877,0.464123
6038,0.429654,0.570346


In [59]:
topk=200
#k=15
t=5     #Represents top-k movies to users
top=10  #Represents Number of user's in a group
ng1=[]
ng2=[]
ng3=[]
ng4=[]
#Sort the membership matrix by group 
srt=Mem_mat.sort_values(by=['Action'], ascending=False)
Top_K=srt.iloc[:topk, :] #select the top k users in a particular group
g1=list(Top_K.T) # getting the indexes of top k users

#selecting top k g2 users
srt=Mem_mat.sort_values(by=['Adventure'], ascending=False)
Top_K=srt.iloc[:topk, :] #select the top k users in a particular group
g2=list(Top_K.T) # getting the indexes of top k users


#selecting top k g3 users
#srt=Mem_mat.sort_values(by=['Animation'], ascending=False)
#Top_K=srt.iloc[:topk, :] #select the top k users in a particular group
#g3=list(Top_K.T) # getting the indexes of top k users

#selecting top k g4 users
#srt=Mem_mat.sort_values(by=['Children's'], ascending=False)
#Top_K=srt.iloc[:topk, :] #select the top k users in a particular group
#g4=list(Top_K.T) # getting the indexes of top k users

#for i in range(k):
#    ng1.append(g1[i])
#    ng2.append(g2[i])
#    ng3.append(g3[i])
#    ng4.append(g4[i])

In [60]:
g1

[1775,
 228,
 3848,
 780,
 2126,
 1780,
 4964,
 3744,
 466,
 1376,
 1114,
 2791,
 3040,
 3921,
 2201,
 3689,
 2199,
 719,
 3296,
 2599,
 4975,
 3663,
 1221,
 4359,
 1991,
 2314,
 1734,
 4698,
 4283,
 2284,
 3765,
 1628,
 2042,
 2881,
 4436,
 4273,
 4694,
 3110,
 2143,
 4645,
 4429,
 3061,
 5687,
 5563,
 3770,
 1059,
 4291,
 3524,
 3799,
 1371,
 1947,
 1978,
 4605,
 6006,
 5253,
 3940,
 3733,
 3878,
 617,
 1121,
 5049,
 3879,
 4324,
 917,
 1433,
 971,
 3152,
 933,
 3967,
 5868,
 4760,
 3793,
 4003,
 5236,
 4854,
 4540,
 1466,
 4058,
 5117,
 737,
 4189,
 5150,
 949,
 1725,
 5491,
 107,
 431,
 4381,
 60,
 2490,
 368,
 3244,
 5416,
 4246,
 3264,
 462,
 3374,
 4440,
 3688,
 2332,
 5745,
 5100,
 4054,
 1317,
 1639,
 4639,
 2796,
 3636,
 4293,
 2225,
 5470,
 4187,
 5298,
 4637,
 3609,
 4684,
 3870,
 2784,
 649,
 3958,
 657,
 745,
 6003,
 5212,
 1944,
 25,
 2738,
 1449,
 1356,
 5958,
 166,
 2826,
 2351,
 3231,
 2624,
 249,
 960,
 5346,
 2781,
 5729,
 5987,
 4208,
 5675,
 1430,
 361,
 798,
 185

In [61]:
r_cols = ["user_id", "movie_id", "rating", "unix_timestamp"]
ratings = pd.read_csv('F:\MS-Research\Group RS\Datasets\ml-1m/ratings.dat', sep='::',  names=r_cols , encoding='ISO-8859-1')

F:\Anaconda3\envs\tf\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [62]:
def groupRat(ratings,g):
    rat_mat=ratings.loc[ratings['user_id']==g[0]]
    for i in range(len(g)-1):
        rat_mat=pd.concat([rat_mat, ratings.loc[ratings['user_id']==g[i+1]]], ignore_index=True)
    #df_combined = pd.merge(rat_mat, movies, on = 'movieId')
    #util_mat = df_combined.pivot_table(index = 'userId', columns = 'movieId', values = 'rating')
    #df=util_mat.fillna(0)
    return rat_mat

In [63]:
ratings=groupRat(ratings, g1) #gives actual ratings and indexes of top k users in particular group
g2_rat=groupRat(ratings, g2)


In [64]:
ratings

,user_id,movie_id,rating,unix_timestamp
0,1775,1250,4,974703452
1,1775,1252,3,974703452
2,1775,2997,3,974701838
3,1775,1259,5,974700744
4,1775,589,3,974700692
...,...,...,...,...
34696,1729,1221,4,981245752
34697,1729,3910,4,981245816
34698,1729,3911,4,974704330
34699,1729,3914,4,974704459


In [65]:
u_cols = ["user_id", "sex", "age_group", "occupation", "zip_code"]
users= pd.read_csv('F:\MS-Research\Group RS\Datasets\ml-1m/users.dat', sep='::', names=u_cols, encoding='ISO-8859-1')

F:\Anaconda3\envs\tf\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [66]:
users=groupRat(users, g1)

In [67]:
users

,user_id,sex,age_group,occupation,zip_code
0,1775,M,25,4,70115
1,228,M,25,15,55455
2,3848,M,25,15,70809-2612
3,780,M,25,17,91776
4,2126,M,56,14,02632
...,...,...,...,...,...
195,2670,M,35,6,94598
196,4845,F,25,1,55413
197,3068,F,35,9,66204
198,973,F,25,1,80026


In [68]:
m_cols=["movie_id", "title", "genres"]
movies = pd.read_csv('F:\MS-Research\Group RS\Datasets\ml-1m/movies.dat', sep='::', names=m_cols, encoding='ISO-8859-1')

F:\Anaconda3\envs\tf\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [69]:
users["user_id"] = users["user_id"].apply(lambda x: f"user_{x}")
users["age_group"] = users["age_group"].apply(lambda x: f"group_{x}")
users["occupation"] = users["occupation"].apply(lambda x: f"occupation_{x}")

movies["movie_id"] = movies["movie_id"].apply(lambda x: f"movie_{x}")

ratings["movie_id"] = ratings["movie_id"].apply(lambda x: f"movie_{x}")
ratings["user_id"] = ratings["user_id"].apply(lambda x: f"user_{x}")
ratings["rating"] = ratings["rating"].apply(lambda x: float(x))

In [70]:
genres = [
    "Action",
    "Adventure",
    "Animation",
    "Children's",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Fantasy",
    "Film-Noir",
    "Horror",
    "Musical",
    "Mystery",
    "Romance",
    "Sci-Fi",
    "Thriller",
    "War",
    "Western",
]

for genre in genres:
    movies[genre] = movies["genres"].apply(
        lambda values: int(genre in values.split("|"))
    )

In [71]:
genres

['Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [72]:
ratings_group = ratings.sort_values(by=["unix_timestamp"]).groupby("user_id")

ratings_data = pd.DataFrame(
    data={
        "user_id": list(ratings_group.groups.keys()),
        "movie_ids": list(ratings_group.movie_id.apply(list)),
        "ratings": list(ratings_group.rating.apply(list)),
        "timestamps": list(ratings_group.unix_timestamp.apply(list)),
    }
)

In [73]:
ratings_group

In [74]:
sequence_length = 4
step_size = 2


def create_sequences(values, window_size, step_size):
    sequences = []
    start_index = 0
    while True:
        end_index = start_index + window_size
        seq = values[start_index:end_index]
        if len(seq) < window_size:
            seq = values[-window_size:]
            if len(seq) == window_size:
                sequences.append(seq)
            break
        sequences.append(seq)
        start_index += step_size
    return sequences


ratings_data.movie_ids = ratings_data.movie_ids.apply(
    lambda ids: create_sequences(ids, sequence_length, step_size)
)

ratings_data.ratings = ratings_data.ratings.apply(
    lambda ids: create_sequences(ids, sequence_length, step_size)
)

del ratings_data["timestamps"]

In [75]:
ratings_data_movies = ratings_data[["user_id", "movie_ids"]].explode(
    "movie_ids", ignore_index=True
)
ratings_data_rating = ratings_data[["ratings"]].explode("ratings", ignore_index=True)
ratings_data_transformed = pd.concat([ratings_data_movies, ratings_data_rating], axis=1)
ratings_data_transformed = ratings_data_transformed.join(
    users.set_index("user_id"), on="user_id"
)
ratings_data_transformed.movie_ids = ratings_data_transformed.movie_ids.apply(
    lambda x: ",".join(x)
)
ratings_data_transformed.ratings = ratings_data_transformed.ratings.apply(
    lambda x: ",".join([str(v) for v in x])
)

del ratings_data_transformed["zip_code"]

ratings_data_transformed.rename(
    columns={"movie_ids": "sequence_movie_ids", "ratings": "sequence_ratings"},
    inplace=True,
)

In [76]:
random_selection = np.random.rand(len(ratings_data_transformed.index)) <= 0.85
train_data = ratings_data_transformed[random_selection]
test_data = ratings_data_transformed[~random_selection]

train_data.to_csv("train_data.csv", index=False, sep="|", header=False)
test_data.to_csv("test_data.csv", index=False, sep="|", header=False)

In [77]:
CSV_HEADER = list(ratings_data_transformed.columns)

CATEGORICAL_FEATURES_WITH_VOCABULARY = {
    "user_id": list(users.user_id.unique()),
    "movie_id": list(movies.movie_id.unique()),
    "sex": list(users.sex.unique()),
    "age_group": list(users.age_group.unique()),
    "occupation": list(users.occupation.unique()),
}

USER_FEATURES = ["sex", "age_group", "occupation"]

MOVIE_FEATURES = ["genres"]

In [78]:
def get_dataset_from_csv(csv_file_path, shuffle=False, batch_size=128):
    def process(features):
        movie_ids_string = features["sequence_movie_ids"]
        sequence_movie_ids = tf.strings.split(movie_ids_string, ",").to_tensor()

        # The last movie id in the sequence is the target movie.
        features["target_movie_id"] = sequence_movie_ids[:, -1]
        features["sequence_movie_ids"] = sequence_movie_ids[:, :-1]

        ratings_string = features["sequence_ratings"]
        sequence_ratings = tf.strings.to_number(
            tf.strings.split(ratings_string, ","), tf.dtypes.float32
        ).to_tensor()

        # The last rating in the sequence is the target for the model to predict.
        target = sequence_ratings[:, -1]
        features["sequence_ratings"] = sequence_ratings[:, :-1]

        return features, target

    dataset = tf.data.experimental.make_csv_dataset(
        csv_file_path,
        batch_size=batch_size,
        column_names=CSV_HEADER,
        num_epochs=1,
        header=False,
        field_delim="|",
        shuffle=shuffle,
    ).map(process)

    return dataset

In [79]:
def create_model_inputs():
    return {
        "user_id": layers.Input(name="user_id", shape=(1,), dtype=tf.string),
        "sequence_movie_ids": layers.Input(
            name="sequence_movie_ids", shape=(sequence_length - 1,), dtype=tf.string
        ),
        "target_movie_id": layers.Input(
            name="target_movie_id", shape=(1,), dtype=tf.string
        ),
        "sequence_ratings": layers.Input(
            name="sequence_ratings", shape=(sequence_length - 1,), dtype=tf.float32
        ),
        "sex": layers.Input(name="sex", shape=(1,), dtype=tf.string),
        "age_group": layers.Input(name="age_group", shape=(1,), dtype=tf.string),
        "occupation": layers.Input(name="occupation", shape=(1,), dtype=tf.string),
    }

In [80]:
def encode_input_features(
    inputs,
    include_user_id=True,
    include_user_features=True,
    include_movie_features=True,
):

    encoded_transformer_features = []
    encoded_other_features = []

    other_feature_names = []
    if include_user_id:
        other_feature_names.append("user_id")
    if include_user_features:
        other_feature_names.extend(USER_FEATURES)

    ## Encode user features
    for feature_name in other_feature_names:
        # Convert the string input values into integer indices.
        vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY[feature_name]
        idx = StringLookup(vocabulary=vocabulary, mask_token=None, num_oov_indices=0)(
            inputs[feature_name]
        )
        # Compute embedding dimensions
        embedding_dims = int(math.sqrt(len(vocabulary)))
        # Create an embedding layer with the specified dimensions.
        embedding_encoder = layers.Embedding(
            input_dim=len(vocabulary),
            output_dim=embedding_dims,
            name=f"{feature_name}_embedding",
        )
        # Convert the index values to embedding representations.
        encoded_other_features.append(embedding_encoder(idx))

    ## Create a single embedding vector for the user features
    if len(encoded_other_features) > 1:
        encoded_other_features = layers.concatenate(encoded_other_features)
    elif len(encoded_other_features) == 1:
        encoded_other_features = encoded_other_features[0]
    else:
        encoded_other_features = None

    ## Create a movie embedding encoder
    movie_vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY["movie_id"]
    movie_embedding_dims = int(math.sqrt(len(movie_vocabulary)))
    # Create a lookup to convert string values to integer indices.
    movie_index_lookup = StringLookup(
        vocabulary=movie_vocabulary,
        mask_token=None,
        num_oov_indices=0,
        name="movie_index_lookup",
    )
    # Create an embedding layer with the specified dimensions.
    movie_embedding_encoder = layers.Embedding(
        input_dim=len(movie_vocabulary),
        output_dim=movie_embedding_dims,
        name=f"movie_embedding",
    )
    # Create a vector lookup for movie genres.
    genre_vectors = movies[genres].to_numpy()
    movie_genres_lookup = layers.Embedding(
        input_dim=genre_vectors.shape[0],
        output_dim=genre_vectors.shape[1],
        embeddings_initializer=tf.keras.initializers.Constant(genre_vectors),
        trainable=False,
        name="genres_vector",
    )
    # Create a processing layer for genres.
    movie_embedding_processor = layers.Dense(
        units=movie_embedding_dims,
        activation="relu",
        name="process_movie_embedding_with_genres",
    )

    ## Define a function to encode a given movie id.
    def encode_movie(movie_id):
        # Convert the string input values into integer indices.
        movie_idx = movie_index_lookup(movie_id)
        movie_embedding = movie_embedding_encoder(movie_idx)
        encoded_movie = movie_embedding
        if include_movie_features:
            movie_genres_vector = movie_genres_lookup(movie_idx)
            encoded_movie = movie_embedding_processor(
                layers.concatenate([movie_embedding, movie_genres_vector])
            )
        return encoded_movie

    ## Encoding target_movie_id
    target_movie_id = inputs["target_movie_id"]
    encoded_target_movie = encode_movie(target_movie_id)

    ## Encoding sequence movie_ids.
    sequence_movies_ids = inputs["sequence_movie_ids"]
    encoded_sequence_movies = encode_movie(sequence_movies_ids)
    # Create positional embedding.
    position_embedding_encoder = layers.Embedding(
        input_dim=sequence_length,
        output_dim=movie_embedding_dims,
        name="position_embedding",
    )
    positions = tf.range(start=0, limit=sequence_length - 1, delta=1)
    encodded_positions = position_embedding_encoder(positions)
    # Retrieve sequence ratings to incorporate them into the encoding of the movie.
    sequence_ratings = tf.expand_dims(inputs["sequence_ratings"], -1)
    # Add the positional encoding to the movie encodings and multiply them by rating.
    encoded_sequence_movies_with_poistion_and_rating = layers.Multiply()(
        [(encoded_sequence_movies + encodded_positions), sequence_ratings]
    )

    # Construct the transformer inputs.
    for encoded_movie in tf.unstack(
        encoded_sequence_movies_with_poistion_and_rating, axis=1
    ):
        encoded_transformer_features.append(tf.expand_dims(encoded_movie, 1))
    encoded_transformer_features.append(encoded_target_movie)

    encoded_transformer_features = layers.concatenate(
        encoded_transformer_features, axis=1
    )

    return encoded_transformer_features, encoded_other_features

In [81]:
include_user_id = False
include_user_features = False
include_movie_features = False

hidden_units = [256, 128]
dropout_rate = 0.1
num_heads = 3


def create_model():
    inputs = create_model_inputs()
    transformer_features, other_features = encode_input_features(
        inputs, include_user_id, include_user_features, include_movie_features
    )

    # Create a multi-headed attention layer.
    attention_output = layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=transformer_features.shape[2], dropout=dropout_rate
    )(transformer_features, transformer_features)

    # Transformer block.
    attention_output = layers.Dropout(dropout_rate)(attention_output)
    x1 = layers.Add()([transformer_features, attention_output])
    x1 = layers.LayerNormalization()(x1)
    x2 = layers.LeakyReLU()(x1)
    x2 = layers.Dense(units=x2.shape[-1])(x2)
    x2 = layers.Dropout(dropout_rate)(x2)
    transformer_features = layers.Add()([x1, x2])
    transformer_features = layers.LayerNormalization()(transformer_features)
    features = layers.Flatten()(transformer_features)

    # Included the other features.
    if other_features is not None:
        features = layers.concatenate(
            [features, layers.Reshape([other_features.shape[-1]])(other_features)]
        )

    # Fully-connected layers.
    for num_units in hidden_units:
        features = layers.Dense(num_units)(features)
        features = layers.BatchNormalization()(features)
        features = layers.LeakyReLU()(features)
        features = layers.Dropout(dropout_rate)(features)

    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


model = create_model()

In [82]:
# Compile the model.
model.compile(
    optimizer=keras.optimizers.Adagrad(learning_rate=0.01),
    loss=keras.losses.MeanSquaredError(),
    metrics=[keras.metrics.MeanAbsoluteError()],
)

# Read the training data.
train_dataset = get_dataset_from_csv("train_data.csv", shuffle=True, batch_size=265)

# Fit the model with the training data.
model.fit(train_dataset, epochs=5)

# Read the test data.
test_dataset = get_dataset_from_csv("test_data.csv", batch_size=265)

# Evaluate the model on the test data.
_, rmse = model.evaluate(test_dataset, verbose=0)
print(f"Test MAE: {round(rmse, 3)}")

Epoch 1/5
56/56 [==============================] - 4s 36ms/step - loss: 2.5889 - mean_absolute_error: 1.2713
Epoch 2/5
56/56 [==============================] - 2s 31ms/step - loss: 1.2415 - mean_absolute_error: 0.8929
Epoch 3/5
56/56 [==============================] - 2s 30ms/step - loss: 1.0991 - mean_absolute_error: 0.8331
Epoch 4/5
56/56 [==============================] - 2s 32ms/step - loss: 0.9839 - mean_absolute_error: 0.7889
Epoch 5/5
56/56 [==============================] - 2s 30ms/step - loss: 0.9058 - mean_absolute_error: 0.7574
Test MAE: 0.885
